In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
#load file
cost_of_living_2_csv = "./Resources/cost-of-living_v2.csv"
inflation_csv = "./Resources/inflation.csv"
Worldcities_csv = "./Resources/worldcities.csv"

In [3]:
#read into pandas dataframes and rename columns
csv2 = pd.read_csv(cost_of_living_2_csv)
worldcities = pd.read_csv(Worldcities_csv)
inflation = pd.read_csv(inflation_csv)
inflation = inflation.rename(columns={'Country': 'country','2021':'2021 inflation rate'})
countries = pd.read_json(r"./Resources/countries.json")
countries = countries.rename(columns={'name': 'country'})
countries

,timezones,latlng,country,country_code,capital
0,[America/Aruba],"[12.5, -69.96666666]",Aruba,AW,Oranjestad
1,[Asia/Kabul],"[33, 65]",Afghanistan,AF,Kabul
2,[Africa/Luanda],"[-12.5, 18.5]",Angola,AO,Luanda
3,[America/Anguilla],"[18.25, -63.16666666]",Anguilla,AI,The Valley
4,[Europe/Mariehamn],"[60.116667, 19.9]",Åland Islands,AX,Mariehamn
...,...,...,...,...,...
242,[Pacific/Apia],"[-13.58333333, -172.33333333]",Samoa,WS,Apia
243,[Asia/Aden],"[15, 48]",Yemen,YE,Sana'a
244,[Africa/Johannesburg],"[-29, 24]",South Africa,ZA,Pretoria
245,[Africa/Lusaka],"[-15, 30]",Zambia,ZM,Lusaka


In [4]:
# concatenate city with country 
worldcities['City, Country'] = worldcities[['city','country']].apply(",".join, axis=1)

In [5]:
csv2 = csv2.rename(columns={'Meal, Inexpensive Restaurant (USD)': 'meal_inexpensive',
    
                                              'Monthly Pass (Regular Price) (USD)': 'monthly_transport_ticket',
                                              'Gasoline (1 liter) (USD)': 'one_liter_gasoline',
                                              'Basic (Electricity, Heating, Cooling, Water, Garbage) for 85m2 Apartment (USD)': 'utilities_915sqft',
                                              'Internet (60 Mbps or More, Unlimited Data, Cable/ADSL) (USD)':'internet',
                                              'Fitness Club, Monthly Fee for 1 Adult (USD)': 'monthly_gym_fee',
                                              'Preschool (or Kindergarten), Full Day, Private, Monthly for 1 Child (USD)': 'monthly_preschool_one_child',
                                              'Apartment (1 bedroom) in City Centre (USD)': 'center_city_1_bd',
                                              'Average Monthly Net Salary (After Tax) (USD)': 'avg_monthly_salary'      
                                    })

csv2['City, Country'] = csv2[['city','country']].apply(",".join, axis=1)
csv2 = csv2.drop('International Primary School, Yearly for 1 Child (USD)', axis=1)
csv2 = csv2.drop('Meal for 2 People, Mid-range Restaurant, Three-course (USD)', axis=1)
csv2 = csv2.drop('One-way Ticket (Local Transport) (USD)', axis=1)
csv2 = csv2.drop('Apartment (1 bedroom) Outside of Centre (USD)', axis=1)
csv2.head()

,city,country,meal_inexpensive,monthly_transport_ticket,one_liter_gasoline,utilities_915sqft,internet,monthly_gym_fee,monthly_preschool_one_child,center_city_1_bd,avg_monthly_salary,"City, Country"
0,Seoul,South Korea,7.68,42.25,1.43,182.13,22.48,55.88,404.15,742.54,2689.62,"Seoul,South Korea"
1,Shanghai,China,5.69,28.47,1.20,66.00,17.07,63.49,1382.62,1091.93,1419.87,"Shanghai,China"
2,Guangzhou,China,4.13,28.47,1.19,59.65,16.66,34.17,555.18,533.28,1211.68,"Guangzhou,China"
3,Mumbai,India,3.68,4.91,1.35,43.57,9.33,25.37,107.07,522.40,640.81,"Mumbai,India"
4,Delhi,India,4.91,11.67,1.24,58.07,7.95,21.20,74.00,229.84,586.46,"Delhi,India"


In [9]:
# add longitude and latitude for each city
city_merged_df = pd.merge(csv2, worldcities, how='inner',on=['City, Country','City, Country']) 
cities_merged_df= city_merged_df[['city_x', 'country_x', 'meal_inexpensive','monthly_transport_ticket','one_liter_gasoline','utilities_915sqft','internet','monthly_gym_fee','monthly_preschool_one_child','center_city_1_bd','avg_monthly_salary','City, Country','lat','lng','population']].copy()
final_cities_merged_df = cities_merged_df.rename(columns={'city_x': 'city', 'country_x':'country'})

In [10]:
# before removing nan
final_cities_merged_df.count()

city                           4903
country                        4903
meal_inexpensive               4432
monthly_transport_ticket       2741
one_liter_gasoline             4244
utilities_915sqft              4481
internet                       4610
monthly_gym_fee                3483
monthly_preschool_one_child    3686
center_city_1_bd               3540
avg_monthly_salary             3418
City, Country                  4903
lat                            4903
lng                            4903
population                     4856
dtype: int64

In [11]:
# after removing some nan information
final_cities_merged_df = final_cities_merged_df[final_cities_merged_df['utilities_915sqft'].notna()]
final_cities_merged_df.count()

city                           4481
country                        4481
meal_inexpensive               4185
monthly_transport_ticket       2685
one_liter_gasoline             4094
utilities_915sqft              4481
internet                       4432
monthly_gym_fee                3351
monthly_preschool_one_child    3618
center_city_1_bd               3404
avg_monthly_salary             3304
City, Country                  4481
lat                            4481
lng                            4481
population                     4440
dtype: int64

In [40]:
#cleaning NaN values
final_cities_merged_df['meal_inexpensive'] = final_cities_merged_df['meal_inexpensive'].fillna(0)
final_cities_merged_df['monthly_transport_ticket'] = final_cities_merged_df['monthly_transport_ticket'].fillna(0)
final_cities_merged_df['one_liter_gasoline'] = final_cities_merged_df['one_liter_gasoline'].fillna(0)
final_cities_merged_df['utilities_915sqft'] = final_cities_merged_df['utilities_915sqft'].fillna(0)
final_cities_merged_df['internet'] = final_cities_merged_df['internet'].fillna(0)
final_cities_merged_df['monthly_gym_fee'] = final_cities_merged_df['monthly_gym_fee'].fillna(0)
final_cities_merged_df['monthly_preschool_one_child'] = final_cities_merged_df['monthly_preschool_one_child'].fillna(0)
final_cities_merged_df['center_city_1_bd'] = final_cities_merged_df['center_city_1_bd'].fillna(0)
final_cities_merged_df['avg_monthly_salary'] = final_cities_merged_df['avg_monthly_salary'].fillna(0)
final_cities_merged_df['lat'] = final_cities_merged_df['lat'].fillna(0)
final_cities_merged_df['lng'] = final_cities_merged_df['lng'].fillna(0)
final_cities_merged_df['population'] = final_cities_merged_df['population'].fillna(0)


#displaying final data for cities
final_cities_merged_df

,city,country,meal_inexpensive,monthly_transport_ticket,one_liter_gasoline,utilities_915sqft,internet,monthly_gym_fee,monthly_preschool_one_child,center_city_1_bd,avg_monthly_salary,"City, Country",lat,lng,population
0,Seoul,South Korea,7.68,42.25,1.43,182.13,22.48,55.88,404.15,742.54,2689.62,"Seoul,South Korea",37.5600,126.9900,22394000.0
1,Shanghai,China,5.69,28.47,1.20,66.00,17.07,63.49,1382.62,1091.93,1419.87,"Shanghai,China",31.1667,121.4667,22118000.0
2,Guangzhou,China,4.13,28.47,1.19,59.65,16.66,34.17,555.18,533.28,1211.68,"Guangzhou,China",23.1288,113.2590,21489000.0
3,Mumbai,India,3.68,4.91,1.35,43.57,9.33,25.37,107.07,522.40,640.81,"Mumbai,India",19.0758,72.8775,22186000.0
4,Delhi,India,4.91,11.67,1.24,58.07,7.95,21.20,74.00,229.84,586.46,"Delhi,India",28.6667,77.2167,31870000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,Tirupati,India,2.46,12.28,1.29,24.93,10.75,15.15,36.85,82.90,614.09,"Tirupati,India",13.6500,79.4167,3165.0
4894,Port Douglas,Australia,15.97,0.00,1.36,129.12,52.33,0.00,1359.16,0.00,5946.32,"Port Douglas,Australia",-16.4834,145.4652,3504.0
4895,Rockhampton,Australia,14.95,91.74,1.43,106.86,42.47,46.21,0.00,781.58,3481.06,"Rockhampton,Australia",-23.3750,150.5117,1953.0
4900,Ixtapa Zihuatanejo,Mexico,5.16,0.00,1.01,77.35,27.46,25.78,0.00,412.55,0.00,"Ixtapa Zihuatanejo,Mexico",17.6367,-101.5514,8992.0


In [13]:
# average the columns from cities dataframe per country - display in new dataframe
world_data_df = final_cities_merged_df[['country', 'meal_inexpensive','monthly_transport_ticket','one_liter_gasoline','utilities_915sqft','internet','monthly_gym_fee','monthly_preschool_one_child','center_city_1_bd','avg_monthly_salary']]
world_data_df = world_data_df .groupby('country').mean()
# adding longitude, latitude, captial and inflation rate for each country
world_merged_df = pd.merge(world_data_df, countries, how='inner',on=['country','country']) 
world_merged2_df = pd.merge(world_merged_df,inflation,how='inner',on=['country','country']) 
world_merged2_df

,country,meal_inexpensive,monthly_transport_ticket,one_liter_gasoline,utilities_915sqft,internet,monthly_gym_fee,monthly_preschool_one_child,center_city_1_bd,avg_monthly_salary,...,2018,2019,2020,2021 inflation rate,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62
0,Afghanistan,1.690000,15.250000,0.890000,31.990000,715.630000,13.560000,25.990000,108.920000,195.860000,...,0.63,2.30,5.44,5.06,NaN,NaN,NaN,NaN,NaN,NaN
1,Albania,4.430000,10.813333,1.760000,91.830000,15.706000,28.847500,105.754000,215.952500,352.356000,...,2.03,1.41,1.62,2.04,NaN,NaN,NaN,NaN,NaN,NaN
2,Algeria,1.942273,10.275385,0.294545,42.192174,39.645000,14.684091,51.689375,136.258000,263.939500,...,3.51,1.95,2.42,7.23,NaN,NaN,NaN,NaN,NaN,NaN
3,Angola,7.840000,31.780000,0.330000,30.780000,192.230000,40.050000,929.370000,220.390000,1428.770000,...,19.63,17.08,21.02,23.85,NaN,NaN,NaN,NaN,NaN,NaN
4,Argentina,6.496923,19.100000,0.877692,53.173077,22.165385,22.597273,164.786923,234.369167,441.514167,...,34.28,53.55,42.02,48.41,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,Uzbekistan,3.696667,10.506000,0.816667,45.048333,16.818333,23.471667,135.630000,205.891667,257.381667,...,17.68,14.53,12.86,10.69,NaN,NaN,NaN,NaN,NaN,NaN
147,Vanuatu,12.710000,NaN,1.440000,285.200000,141.900000,67.470000,544.080000,995.460000,677.680000,...,2.31,2.75,2.92,2.39,NaN,NaN,NaN,NaN,NaN,NaN
148,Vietnam,1.956000,8.296667,1.014000,59.348000,11.264000,27.460000,266.704000,396.828000,406.477500,...,3.54,2.80,3.22,1.83,NaN,NaN,NaN,NaN,NaN,NaN
149,Zambia,3.000000,38.640000,1.360000,140.690000,103.740000,23.820000,171.240000,262.010000,396.940000,...,7.49,9.80,15.73,22.02,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
final_world_merged_df= world_merged2_df [['country', 'meal_inexpensive','monthly_transport_ticket','one_liter_gasoline','utilities_915sqft','internet','monthly_gym_fee','monthly_preschool_one_child','center_city_1_bd','avg_monthly_salary','latlng','capital','2021 inflation rate']]

final_world_merged_df

,country,meal_inexpensive,monthly_transport_ticket,one_liter_gasoline,utilities_915sqft,internet,monthly_gym_fee,monthly_preschool_one_child,center_city_1_bd,avg_monthly_salary,latlng,capital,2021 inflation rate
0,Afghanistan,1.690000,15.250000,0.890000,31.990000,715.630000,13.560000,25.990000,108.920000,195.860000,"[33, 65]",Kabul,5.06
1,Albania,4.430000,10.813333,1.760000,91.830000,15.706000,28.847500,105.754000,215.952500,352.356000,"[41, 20]",Tirana,2.04
2,Algeria,1.942273,10.275385,0.294545,42.192174,39.645000,14.684091,51.689375,136.258000,263.939500,"[28, 3]",Algiers,7.23
3,Angola,7.840000,31.780000,0.330000,30.780000,192.230000,40.050000,929.370000,220.390000,1428.770000,"[-12.5, 18.5]",Luanda,23.85
4,Argentina,6.496923,19.100000,0.877692,53.173077,22.165385,22.597273,164.786923,234.369167,441.514167,"[-34, -64]",Buenos Aires,48.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,Uzbekistan,3.696667,10.506000,0.816667,45.048333,16.818333,23.471667,135.630000,205.891667,257.381667,"[41, 64]",Tashkent,10.69
147,Vanuatu,12.710000,NaN,1.440000,285.200000,141.900000,67.470000,544.080000,995.460000,677.680000,"[-16, 167]",Port Vila,2.39
148,Vietnam,1.956000,8.296667,1.014000,59.348000,11.264000,27.460000,266.704000,396.828000,406.477500,"[16.16666666, 107.83333333]",Hanoi,1.83
149,Zambia,3.000000,38.640000,1.360000,140.690000,103.740000,23.820000,171.240000,262.010000,396.940000,"[-15, 30]",Lusaka,22.02


# connect to local database

In [41]:
import psycopg2

In [42]:
from sqlalchemy import create_engine

In [43]:
protocol = 'postgresql'
username = 'postgres'
password = 'vineyard'
host = 'localhost'
port = 5432
database_name = 'inflation_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [44]:
engine.table_names()

C:\Users\Elizabeth\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['countries', 'cities']

In [46]:
# load cities data
final_cities_merged_df.to_sql(name='cities', con=engine, if_exists='append', index=False)

In [47]:
# load cities data
final_world_merged_df.to_sql(name='countries', con=engine, if_exists='append', index=False)